In [1]:
import pandas as pd
df=pd.read_csv("./Datasets/mobiles.csv")
df.head()

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39
2,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
3,Medium,64,3,1,1,2942,4.6,5366,42999,0.10,23.07
4,Medium,128,4,2,1,2815,4.6,745,69149,0.02,5.15


In [23]:
mean=df['sales'].mean()
std = df['sales'].std()
outlier= mean+2*std
print(outlier)
df1=df.loc[df['sales']>outlier,]
df1['performance'] = (df1.ROM/32)+(df1.RAM/2)+df1.num_rear_camera+df1.num_front_camera+(df1.battery_capacity/1000)
round(df1['performance'].mean(),2)

146.5515012927322


<ipython-input-23-fe45f4aa4a39>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['performance'] = (df1.ROM/32)+(df1.RAM/2)+df1.num_rear_camera+df1.num_front_camera+(df1.battery_capacity/1000)


11.01

A1:11.01

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [6]:
df1=df[df.num_rear_camera>1]
df1.head(10)
df1=df1[['battery_capacity','ratings','num_of_ratings','sales_price','discount_percent','sales']]
d_cor=df1.corr()
d_cor['abs_sales']=abs(d_cor.sales)
d_cor.head()
d_cor=d_cor.sort_values(by='abs_sales',ascending=False)
round(d_cor['abs_sales'][1],2)

0.95

A2: 0.95

In [16]:
df3=df.copy()
df3.head(5)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39
2,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
3,Medium,64,3,1,1,2942,4.6,5366,42999,0.10,23.07
4,Medium,128,4,2,1,2815,4.6,745,69149,0.02,5.15


In [13]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   screen_size       430 non-null    object 
 1   ROM               430 non-null    int64  
 2   RAM               430 non-null    int64  
 3   num_rear_camera   430 non-null    int64  
 4   num_front_camera  430 non-null    int64  
 5   battery_capacity  430 non-null    int64  
 6   ratings           430 non-null    float64
 7   num_of_ratings    430 non-null    int64  
 8   sales_price       430 non-null    int64  
 9   discount_percent  430 non-null    float64
 10  sales             430 non-null    float64
dtypes: float64(3), int64(7), object(1)
memory usage: 37.1+ KB


In [17]:
df3 = pd.get_dummies(df3,columns=['screen_size'])
df3.columns

Index(['ROM', 'RAM', 'num_rear_camera', 'num_front_camera', 'battery_capacity',
       'ratings', 'num_of_ratings', 'sales_price', 'discount_percent', 'sales',
       'screen_size_Large', 'screen_size_Medium', 'screen_size_Small',
       'screen_size_Very Large', 'screen_size_Very Small'],
      dtype='object')

In [26]:
col_X = ['ROM', 'RAM', 'num_rear_camera', 'num_front_camera', 'battery_capacity',
       'ratings', 'num_of_ratings', 'sales_price', 'discount_percent', 
       'screen_size_Large', 'screen_size_Medium', 'screen_size_Small',
       'screen_size_Very Large', 'screen_size_Very Small']
X = df3[col_X]
y = df3['sales'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [27]:
scaler_mm = MinMaxScaler()
X_train=scaler_mm.fit_transform(X_train)
print(X_train.shape)
X_test=scaler_mm.transform(X_test)
print(X_test)

(344, 14)
[[0.49206349 0.27272727 0.33333333 ... 0.         0.         0.        ]
 [0.04761905 0.09090909 0.33333333 ... 0.         0.         0.        ]
 [0.49206349 1.         0.66666667 ... 0.         0.         0.        ]
 ...
 [0.23809524 0.27272727 0.33333333 ... 1.         0.         0.        ]
 [0.23809524 0.63636364 1.         ... 0.         0.         0.        ]
 [0.04761905 0.18181818 0.66666667 ... 0.         0.         0.        ]]


In [39]:
k=[3,4,7,9,11]
mse={}
for i in k:
    model=KNeighborsRegressor(n_neighbors=i)
    model.fit(X=X_train,y=y_train)
    y_pre = model.predict(X_test)
    m=mean_squared_error(y_true = y_test,
                   y_pred = y_pre)
    mse[i]=m
print(mse)
sorted(mse.items(),key=lambda x: x[1],reverse=True)[0]

{3: 1635.4379954780363, 4: 2087.688149781977, 7: 2828.8309384907457, 9: 3078.5168532443304, 11: 3154.0245733999623}


(11, 3154.0245733999623)

In [31]:
min(mse)

1635.4379954780363